In [1]:
import csv
import pandas
import urllib
import json
import time

In [ ]:
with open('songdata.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    contents = []
    for row in f_csv:
        contents.append(row)
print(contents[10])
    

In [ ]:
'''API_VERSION = '2'
HOST_NAME = 'https://itunes.apple.com/search?'

class Search(object):
    def __init__(self, query, lang = 'en_us', version=API_VERSION, explicit='Yes'):
        self._search_terms = dict()
        self._search_terms['term'] = query
        self._search_terms['lang'] = lang
        self._search_terms['explicit'] = explicit
        
        self._json_results = None
        self._num_results = None
    
    def num_results(self):
        return self._num_results
    
    
class Request(object):
    def __init__(self, params):
        self.params = params
        
    def _download_response(self):
        data = []
        for item in self.params:
            temp = item.split(' ')
            temp = '+'.join(temp)
            data.append('term=' + temp)
        data = '&'.join(data)
        
        url = HOST_NAME+data
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
        return response.read()
    
    def execute(self, cacheable=False):
        try:
            response = self._download_response()
            return json.loads(response)
        except urllib.error.HTTPError as e:
            print (e.fp.read())
            return 0
    
    def _get_error(self, text):
        return ServiceException(type='Error', message=text)
        raise
    
class ServiceException(Exception):
    """Exception related to the web service."""

    def __init__(self, type, message):
        self._type = type
        self._message = message

    def __str__(self):
        return self._type + ': ' + self._message

    def get_message(self):
        return self._message

    def get_type(self):
        return self._type
    
    
def genre(params):
    if Request(params).execute() == 0:
        time.sleep(20)
        genre(params)
        return 'None 1'
    else:
        jsonfile = Request(params).execute()
    if len(jsonfile['results']) == 0:
        return 'None 2'
    else:
        return jsonfile['results'][0]['primaryGenreName']
    
def genre2(params):
    jsonfile = Request2(params).execute()

for i in range(0,len(contents)):
    time.sleep(0.1)
    print(str(i), "song: ",genre([contents[i][0],contents[i][1]]))'''

In [ ]:
api_key = 'b561489917812d8f71d28c4c51e62894'
HOST_name = 'http://ws.audioscrobbler.com/2.0/?method=track.getInfo&'

class Request2(object):
    def __init__(self, params):
        self.params = params
        
    def _download_response(self):
        data = []
        for i in range(len(self.params)):
            item = self.params[i]
            if i ==0:
                temp = item.split(' ')
                temp = '+'.join(temp)
                data.append('artist=' + temp)
            if i ==1:
                temp = item.split(' ')
                temp = '+'.join(temp)
                data.append('track=' + temp)
        data = '&'.join(data)
        
        url = HOST_name+"api_key="+api_key+'&'+data+"&format=json"
        request = urllib.request.Request(url)
        response = urllib.request.urlopen(request)
        return response.read()
    
    def execute(self, cacheable=False):
        try:
            response = self._download_response()
            return json.loads(response)
        except urllib.error.HTTPError as e:
            print (e.fp.read())
            return 0
        
def genre2(params):
    jsonfile = Request2(params).execute()
    if jsonfile.get('error'):
        return "None"
    if len(jsonfile['track']['toptags']['tag'])>0:
        return jsonfile['track']['toptags']['tag'][0]['name']
    else:
        return "None"


# for i in range(0,len(contents)):
with open("songcat1.csv", "w+") as f:
    writer = csv.writer(f)
    header = ['Sr.No', 'artist','song', 'cat']
    writer.writerows([header])
    for i in range(40000, len(contents)):
        time.sleep(1)
        string = str(i) + "\t" + contents[i][0] + "\t"+ contents[i][1] + "\t"
        string += genre2([contents[i][0],contents[i][1]])
#         print(string.split('\t'))
        writer.writerows([string.split('\t')])
print('finish\n')

In [ ]:
filenames = ['songcat10000.csv','songcat20000.csv','songcat30000.csv']
with open('songcat30000summary.csv','w') as f:
    writer = csv.writer(f)
    header = ['Sr.No', 'artist','song', 'cat']
    writer.writerows([header])
    for file in filenames:
        with open(file) as f1:
            f_csv = csv.reader(f1)
            headers = next(f_csv)
            for row in f_csv:
                writer.writerows([row])

Examples to start the bag of words model for feature extraction

In [2]:
#################classification Bag of words model by Mingmin Song
#This is to build vocabulary 
import string
from scipy import sparse
import numpy as np

def buildOrdDict(filename): 
    '''
    filename is lyric filename
    '''
    import pandas as pd
    df = pd.read_csv(filename)
    labels = list(df['cat'])
    cats   = list(set(labels))
    print(cats)
#     cats = ['rock', 'pop', 'country', 'jazz','folk', 'metal']
    voc = set()
    ordered_set={}
    contents = []
    labels = []
    with open(filename) as f:
        f_csv = csv.reader(f)
        headers = next(f_csv)
        for row in f_csv:
            lyrics = row[2].replace('\n',',')
            label = cats.index(row[3])
            table = str.maketrans({key: None for key in string.punctuation})
            words = [word.lower() for word in lyrics.translate(table).split()]
            contents.append(words)
            labels.append(label)
            voc.update(words)
        
    for i,word in enumerate(voc):
        ordered_set[word] = i
    return contents, labels, ordered_set

def buildFeatureVectorX(Voc, Contents, Labels):
    '''
    Voc is the ordered set, Contents is a list of processed lyric list
    '''
    rows, cols, data = [],[],[]
    local_cols = {}
    for i, content in enumerate(Contents):
        for word in content:
            index = Voc[word]
            local_cols.setdefault(index, 0)
            local_cols[index] += 1
    for col in local_cols.keys():
        rows.append(i)
        cols.append(col)
        data.append(local_cols[col])

    X=sparse.coo_matrix((np.array(data),(np.array(rows),np.array(cols))),shape=(len(Contents),len(Voc)))
#     sparse.save_npz('train_X.npz', X)
    Y=np.array(Labels)
    return X,Y

In [3]:
filename = 'songcleaned2.csv'
contents, labels, ordered_voc = buildOrdDict(filename)

['jazz', 'folk', 'rock', 'country', 'metal']


In [4]:
from collections import Counter
print(Counter(labels))

Counter({2: 6804, 0: 6015, 1: 5810, 3: 5680, 4: 5016})


In [5]:
X, Y = buildFeatureVectorX(ordered_voc, contents, labels)

In [6]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
method1_3="Setting1: Data Splitting, Naive Bayes"
gnb = MultinomialNB()
gnb.fit(X_train, y_train)
pred_gnb_setting1 = gnb.predict(X_test)

dtree = tree.DecisionTreeClassifier(random_state=0)
dtree.fit(X_train, y_train)
pred_dtree_setting1 = dtree.predict(X_test)

In [7]:
def Accuracy_Precision_Recall(y_test, predicted):
    #overall accuracy
    overall_accuracy = accuracy_score(y_test, predicted)
    print('\tOverall accuracy' + str(overall_accuracy))

In [8]:
Accuracy_Precision_Recall(y_test, pred_gnb_setting1)
Accuracy_Precision_Recall(y_test, pred_dtree_setting1)

	Overall accuracy0.229824960218
	Overall accuracy0.229824960218


In [ ]:
print(len(ordered_voc))

In [125]:
import pandas as pd
df1 = pd.read_csv('songdata.csv')

In [126]:
df2 = pd.read_csv('songcat.csv',encoding = "ISO-8859-1")

In [127]:
result = pd.concat([df1['artist'], df1['song'], df1['text'], df2['cat']], axis=1)

In [135]:
import random
cats = ['rock', 'pop', 'country', 'jazz','folk', 'metal']
res  = []
# temp = result[result['cat'].str.contains('rock')]
# temp['cat'] = 'rock'
remove_n = 6000
for cat in cats:
    temp = result[result['cat'].str.contains(cat)]
    temp['cat'] = cat
    if cat == 'rock':
        drop_indices = np.random.choice(temp.index, remove_n, replace=False)
        df_subset = temp.drop(drop_indices)
        res.append(df_subset)
    elif cat == 'country' or cat == 'metal':
        res.append(temp)
        res.append(temp)
    elif cat == 'jazz' or cat == 'folk':
        res.append(temp)
        res.append(temp)
        res.append(temp)
        res.append(temp)
        res.append(temp)
#     res.append(result.loc[result['cat']==cat])
df = pd.concat(res)

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [136]:
df.to_csv('songcleaned2.csv', sep=',', index=False)

In [3]:
#######################################################
import pandas as pd
df = pd.read_csv('songcleaned.csv')

In [16]:
labels = list(df['cat'])
cats   = list(set(labels))

In [17]:
for i, label in enumerate(labels):
    index = cats.index(label)
    labels[i] = index

In [19]:
import numpy as np
import pickle
pickle.dump(np.array(labels), open('labels.p', 'wb'))

In [137]:
df = pd.read_csv('songcleaned2.csv')

In [138]:
# df.drop(df.index[0:5000], inplace=True)
# df.to_csv('songcleaned2.csv', sep=',', index=False)

In [139]:
df[df['cat']=='rock']

,artist,song,text,cat
0,Adam Sandler,Love Stinks,Whooo hoo \nCindy and Scott are newley-weds ...,rock
1,Aerosmith,Draw The Line,"Checkmate honey, beat ya at your own damn game...",rock
2,Aerosmith,Eat The Rich,Well I woke up this morning \nOn the wrong si...,rock
3,Aerosmith,Janie's Got A Gun,"Dum, dum, dum, honey what have you done \nDum...",rock
4,Aerosmith,Lay It Down,Ruby red... her lips were on fire \nA do me w...,rock
5,Aerosmith,Love Me Two Times,Ah yeah \nLove me two time baby \nLove me tw...,rock
6,Aerosmith,Luv Lies,Oh... when you cross that line that you know y...,rock
7,Aerosmith,Magic Touch,"Somebody better call a doctor, doctor \nOr wa...",rock
8,Aerosmith,Mia,Hush-a-bye my baby soft and new \nOoh her lov...,rock
9,Aerosmith,Milkcow Blues,Well I tried everything \nJust to get along w...,rock
